# DAILY GLOBAL HISTORICAL CLIMATOLOGY NETWORK (GHCN-DAILY) 

Dense and regression tasks

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import shutil
import sys
sys.path.append('../..')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to chosen GPU to use, nothing if work on CPU

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import cartopy.crs as ccrs

In [ ]:
from deepsphere import models, experiment_helper, plot, utils

In [ ]:
from GHCN_preprocessing import get_data, get_stations, sphereGraph, clean_nodes

In [ ]:
import numpy as np
from scipy import sparse

def regression_tikhonov(G, y, M, tau=0):
    r"""Solve a regression problem on graph via Tikhonov minimization.
    The function solves
    .. math:: \operatorname*{arg min}_x \| M x - y \|_2^2 + \tau \ x^T L x
    if :math:`\tau > 0`, and
    .. math:: \operatorname*{arg min}_x x^T L x \ \text{ s. t. } \ y = M x
    otherwise.
    Parameters
    ----------
    G : :class:`pygsp.graphs.Graph`
    y : array, length G.n_vertices
        Measurements.
    M : array of boolean, length G.n_vertices
        Masking vector.
    tau : float
        Regularization parameter.
    Returns
    -------
    x : array, length G.n_vertices
        Recovered values :math:`x`.
    Examples
    --------
    >>> from pygsp import graphs, filters, learning
    >>> import matplotlib.pyplot as plt
    >>>
    >>> G = graphs.Sensor(N=100, seed=42)
    >>> G.estimate_lmax()
    Create a smooth ground truth signal:
    >>> filt = lambda x: 1 / (1 + 10*x)
    >>> filt = filters.Filter(G, filt)
    >>> rs = np.random.RandomState(42)
    >>> signal = filt.analyze(rs.normal(size=G.n_vertices))
    Construct a measurement signal from a binary mask:
    >>> mask = rs.uniform(0, 1, G.n_vertices) > 0.5
    >>> measures = signal.copy()
    >>> measures[~mask] = np.nan
    Solve the regression problem by reconstructing the signal:
    >>> recovery = learning.regression_tikhonov(G, measures, mask, tau=0)
    Plot the results:
    >>> fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(10, 3))
    >>> limits = [signal.min(), signal.max()]
    >>> _ = G.plot_signal(signal, ax=ax1, limits=limits, title='Ground truth')
    >>> _ = G.plot_signal(measures, ax=ax2, limits=limits, title='Measures')
    >>> _ = G.plot_signal(recovery, ax=ax3, limits=limits, title='Recovery')
    >>> _ = fig.tight_layout()
    """

    if tau > 0:

        y[M == False] = 0

        if sparse.issparse(G.L):

            def Op(x):
                return (M * x.T).T + tau * (G.L.dot(x))

            LinearOp = sparse.linalg.LinearOperator([G.N, G.N], Op)

            if y.ndim > 1:
                sol = np.empty(shape=y.shape)
                res = np.empty(shape=y.shape[1])
                for i in range(y.shape[1]):
                    sol[:, i], res[i] = sparse.linalg.cg(
                        LinearOp, y[:, i])
            else:
                sol, res = sparse.linalg.cg(LinearOp, y)

            # TODO: do something with the residual...
            return sol

        else:

            # Creating this matrix may be problematic in term of memory.
            # Consider using an operator instead...
            if type(G.L).__module__ == np.__name__:
                LinearOp = np.diag(M*1) + tau * G.L
            return np.linalg.solve(LinearOp, M * y)

    else:

        if np.prod(M.shape) != G.n_vertices:
            raise ValueError("M should be of size [G.n_vertices,]")

        indl = M
        indu = (M == False)

        Luu = G.L[indu, :][:, indu]
        Wul = - G.L[indu, :][:, indl]

        if sparse.issparse(G.L):
            sol_part = sparse.linalg.spsolve(Luu, Wul.dot(y[indl]))
        else:
            sol_part = np.linalg.solve(Luu, np.matmul(Wul, y[indl]))

        sol = y.copy()
        sol[indu] = sol_part

        return sol

In [ ]:
datapath = "/mnt/nas/LTS2/datasets/ghcn-daily/processed/"
rawpath = "/mnt/nas/LTS2/datasets/ghcn-daily/raw/"
newdatapath = "./data/ghcn-daily/processed/"

In [ ]:
years = np.arange(2010,2015)
years2 = np.arange(2010,2018)

feature_names = ['PRCP', 'TMIN', 'TMAX', 'SNOW', 'SNWD', 'WT']
# max stations, 50'469
# in 2010, in nbr stations [36k, 16k, 16k, 6k, 22k, (20k, 0, 0, 28, 1k, 6), 5k]
# in 2011, in nbr stations [36k, 16k, 16k, 6k, 22k, (18k, 0, 0, 38, 1k, 22), 5k]
# in 2014, in nbr stations [39k, 15k, 15k, 6k, 24k, (19k, 0, 0, 40, 1k, 49), 4k]
n_features = len(feature_names)
n_years  = len(years)

## Fetch data

In [ ]:
n_stations, ghcn_to_local, lat, lon, alt, _ = get_stations(datapath, years)
full_data, n_days, valid_days = get_data(newdatapath, years, feature_names, ghcn_to_local)

In [ ]:
assert n_stations == full_data.shape[0]

print(f'n_stations: {n_stations}, n_days: {n_days}')

In [ ]:
neighbour = 10

## Preprocess data

#### Temperature MAX from Temperature MIN

In [ ]:
EXP = 'temp'
datas_temp, keep_temp, gTemp = clean_nodes(full_data, [1,3], lon, lat, figs=True, rad=False)

In [ ]:
min(gTemp.dw)

In [ ]:
plt.hist(gTemp.dw, )

In [ ]:
leap_years = np.zeros_like(years).astype(np.bool)
for i, in_year in enumerate(np.split(valid_days,len(years))):
    leap_years[i] = in_year.sum()==366

In [ ]:
w_months = np.tile(np.repeat(np.arange(12), 31), years[-1]-years[0]+1)[valid_days]
w_days = np.tile(np.arange(365),years[-1]-years[0]+1)
for i, leap in enumerate(leap_years):
    if leap:
        w_days = np.insert(w_days, ((i+1)*365), 365)

In [ ]:
w_days_sin = np.sin(w_days/367*np.pi)
w_days_cos = -np.cos(w_days/367*np.pi*2)/2+0.5

In [ ]:
from GHCN_preprocessing import dataset_temp
training, validation = dataset_temp(datas_temp, lon[keep_temp], lat[keep_temp], alt[keep_temp], w_days, add_feat=True)

###### Super set

In [ ]:
EXP += '_superset'
datas_super, keepSuper, gTempSuper = clean_nodes(full_data, [1,3], lon, lat, superset=True, figs=False, rad=False, epsilon=False)
mask = ~np.isnan(dataset_super)
gTempSuper.compute_laplacian("combinatorial")

In [ ]:
min(gTempSuper.dw)

In [ ]:
plt.hist(gTempSuper.dw)

In [ ]:
for day in range(dataset_super.shape[0]):
    for feat in range(dataset_super.shape[-1]):
        dataset_super[day,:,feat] = regression_tikhonov(gTempSuper, dataset_super[day,:,feat], mask[day,:,feat])

In [ ]:
training, validation = dataset_temp(dataset_super, lon[keepSuper], lat[keepSuper], alt[keepSuper], w_days, add_feat=True)

In [ ]:
plt.scatter(dataset_super[:,:,0], dataset_super[:,:,1])

In [ ]:
np.where(dataset_super[:,:,0]> dataset_super[:,:,1])[0]

#### Precipirtation from temperatures (MIN, MAX)

In [ ]:
EXP = 'prec'
datas_prec, keep_prec, gPrec = clean_nodes(full_data, [0,3], lon, lat, figs=True, rad=False)

In [ ]:
from GHCN_preprocessing import dataset_prec
training, validation = dataset_prec(datas_prec, lon[keep_prec], lat[keep_prec], alt[keep_prec], w_days, add_feat=False)

In [ ]:
x_train, _ = training.get_all_data()
time = np.empty_like(x_train[:,:,0])
time[:,:] = np.arange(time.shape[0])[:,np.newaxis]

#### Find future temperature

In [ ]:
EXP = 'future'
datas_temp_reg, keep_reg, gReg = clean_nodes(full_data, [1,3], lon, lat, figs=True, rad=False)

In [ ]:
from GHCN_preprocessing import dataset_reg
training, validation = dataset_reg(datas_temp_reg, lon[keep_reg], lat[keep_reg], alt[keep_reg], w_days_sin, add_feat=False)

In [ ]:
placereg = np.empty_like(datas_temp_reg[:,:,0])
placereg[:,:] = np.arange(placereg.shape[1])

In [ ]:
## is it possible to have a difference of 40 degrees between 1 day?  (must fix to +- 30 degrees)
plt.scatter(x_train[:,:,days_pred-1], labels_train)
plt.plot(range(-50, 45), range(-50, 45),'r')

#### dense-classification, segmentation weather types

In [ ]:
EXP = 'seg'
datas_seg, keep_seg, gSeg = clean_nodes(full_data, [-1,None], lon, lat, figs=False, superset=True, rad=False)

In [ ]:
filt_data = []
for i in range(1,3):
    filt_data.append(datas_seg[:,:,i][~np.isnan(datas_seg[:,:,i])].flatten())
plt.boxplot(filt_data)

In [ ]:
dict_label = {1: 'fog', 2: 'heavy fog', 3: 'thunder', 4: 'ice pellets', 5: 'hail', 6: 'rime ice', 7: 'volcanic ash',
              8: 'smoke', 9: 'blowing snow', 10: 'tornado', 11: 'high winds', 12: 'blowing spray', 13: 'mist',
              14: 'drizzle', 15: 'freezing drizzle', 16: 'rain', 17: 'freezing rain', 18: 'snow', 19: 'unkown prec',
              21: 'ground fog', 22: 'ice fog'}

In [ ]:
fog = datas_seg[np.where(datas_seg[:,:,-1]==4)]

In [ ]:
filt_fog = []
for i in range(1,3):
    filt_fog.append(fog[:,i][~np.isnan(fog[:,i])])
plt.boxplot(filt_fog)

#### snow prevision

In [ ]:
EXP = 'snow'
datas_snow, keep_snow, gSnow = clean_nodes(full_data, [0,5], lon, lat, figs=True, rad=False)

In [ ]:
from GHCN_preprocessing import dataset_snow
training, validation = dataset_prec(datas_snow, lon[keep_snow], lat[keep_snow], alt[keep_snow], w_days_sin, add_feat=False)

#### Global regression task

In [ ]:
EXP = 'global_reg'
plt.plot(w_days)

In [ ]:
plt.plot(w_days_sin)

In [ ]:
plt.plot(w_days_cos)

In [ ]:
from GHCN_preprocessing import dataset_global
training, validation = dataset_global(datas_prec, lon[keep_prec], lat[keep_prec], alt[keep_prec], w_days_cos, add_feat=False)

## training

In [ ]:
from GHCN_train import hyperparameters_dense, hyperparameters_global

In [ ]:
nfeat = training.get_all_data()[0].shape[-1]
if 'global' in EXP:
    params = hyperparameters_global(gPrec, EXP, neighbour, nfeat, training.N)
else:
    params = hyperparameters_dense(gReg, EXP, neighbour, nfeat, training.N)
if 'super' in EXP:
    params['mask'] = [mask[:limit,:,1], mask[limit:,:,1]]
model = models.cgcnn(**params)

In [ ]:
shutil.rmtree('summaries/{}/'.format(EXP_NAME), ignore_errors=True)
shutil.rmtree('checkpoints/{}/'.format(EXP_NAME), ignore_errors=True)

In [ ]:
accuracy_validation, loss_validation, loss_training, t_step, t_batch = model.fit(training, validation)

In [ ]:
plot.plot_loss(loss_training, loss_validation, t_step, params['eval_frequency'])

In [ ]:
x_val, labels_val = validation.get_all_data()
res = model.predict(x_val)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
def mre(labels, predictions):
    return np.mean(np.abs((labels - predictions) / np.clip(labels, 1, None))) * 100

In [ ]:
res = model.predict(np.atleast_3d(x_val))

In [ ]:
if 'global' not in EXP:
    mse_ = (mean_squared_error(labels_val[:-1,:], res[1:,:]))
    mae_ = (mean_absolute_error(labels_val[:-1,:], res[1:,:]))
    mre_ = (mre(labels_val[:-1,:], res[1:,:]))
    r2_ = (r2_score(labels_val[:-1,:], res[1:,:]))
    expvar_ = (explained_variance_score(labels_val[:-1,:], res[1:,:]))
    print("MSE={:.2f}, MAE={:.2f}, MRE={:.2f}, R2={:.3f}, Expvar={:.4f}".format(mse_, mae_, mre_, r2_, expvar_))

In [ ]:
if 'future' in EXP:
    predictions = []
    for i in range(days_pred):
        predictions.append(dataset_temp_reg[i,:,0])
    for i in tqdm(range(len(dataset_temp_reg)-2*days_pred)):
        x_pred = np.asarray(predictions[-days_pred:]).T
        x_pred = np.hstack([x_pred, 
    #                      np.broadcast_to(w_months[np.newaxis,i:i+days_pred], x_pred.shape),
                           coords_v,
                           alt_v[:,np.newaxis],
                           np.repeat(w_days_sin[i], x_pred.shape[0], axis=0)[:,np.newaxis]])
                           #np.broadcast_to(w_days[np.newaxis, i:i+days_pred], x_pred.shape)])
        x_pred = np.repeat(x_pred[np.newaxis,:,:], 64, axis=0)
        res = model.predict(x_pred)
        predictions.append(res[0,:])
        
    plt.scatter(timereg[days_pred:,:], np.asarray(predictions))

In [ ]:
res = model.predict(np.atleast_3d(x_val))
plt.scatter(x_val[:,:,0], res)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
mse_loc = []
mae_loc = []
for i in range(res.shape[1]):
#     mse_loc.append(mse(dataset_temp[:,i,1], res[:,i]))
#     mae_loc.append(mae(dataset_temp[:,i,1], res[:,i]))
#     mse_loc.append(mse(labels_train[:,i]*mask[:limit,i,1], res[:,i]*mask[:limit,i,1]))
#     mae_loc.append(mae(labels_train[:,i]*mask[:limit,i,1], res[:,i]*mask[:limit,i,1]))
    mse_loc.append(mse(labels_val[:,i], res[:,i]))
    mae_loc.append(mae(labels_val[:,i], res[:,i]))
plt.plot(mse_loc)
plt.figure()
plt.plot(mae_loc)

In [ ]:
mse_tab = []
mae_tab = []
for i in range(res.shape[0]-1):
#     mse_tab.append(mse(dataset_temp[i,:,1], res[i,:]))
#     mse_tab.append(mse(labels_train[i,:]*mask[i,:,1], res[i,:]*mask[i,:,1]))
    mse_tab.append(mse(labels_val[i,:], res[i,:]))
    mae_tab.append(mae(labels_val[i,:], res[i,:]))
plt.plot(mse_tab)
plt.figure()
plt.plot(mae_tab)

In [ ]:
arg = np.argmax(mse_tab)

In [ ]:
mse_loc = []
mae_loc = []
index = []
for i in range(res.shape[1]):
    mse_loc.append(mse([labels_val[arg,i]], [res[arg,i]]))
    mae_loc.append(mae([labels_val[arg,i]], [res[arg,i]]))
    if mae_loc[-1]>7:
        index.append(i)
plt.plot(mse_loc)
plt.figure()
plt.plot(mae_loc)
index = np.asarray(index)

In [ ]:
fig = plt.figure(figsize=(30, 50))
ax = fig.add_subplot(3, 2, 1, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('labels day D')

zmin, zmax = 0, 40

sc = ax.scatter(lon[keep], lat[keep], s=10,
                c=np.clip(labels_val[arg, :], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
#                 c=np.clip(labels_train[arg, :]*masknan[arg, :, 1], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

ax = fig.add_subplot(3, 2, 2, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('prediction day D')


sc = ax.scatter(lon[keep], lat[keep], s=10,
                c=np.clip(res[arg, :], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
#                 c=np.clip(res[arg, :]*masknan[arg, :, 1], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

ax = fig.add_subplot(3, 2, 4, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('prediction day D+1')

sc = ax.scatter(lon[keep], lat[keep], s=10,
                c=np.clip(res[arg+1, :], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
#                 c=np.clip(res[arg+1, :]*masknan[arg+1, :, 1], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

# ax = fig.add_subplot(3, 2, 5, projection=ccrs.Orthographic(-50, 90))
# ax.set_global()
# ax.coastlines(linewidth=2)
# ax.set_title('given D day D-1')


# sc = ax.scatter(lon[keepToo], lat[keepToo], s=10,
#                 c=np.clip(x_train[arg, :, days_pred-1], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
#                 vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

ax = fig.add_subplot(3, 2, 3, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('labels day D+1')


sc = ax.scatter(lon[keep], lat[keep], s=10,
                c=np.clip(labels_val[arg+1, :], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

# ax = fig.add_subplot(3, 2, 6, projection=ccrs.Orthographic(-50, 90))
# ax.set_global()
# ax.coastlines(linewidth=2)
# ax.set_title('given D+1 day D-1')

# sc = ax.scatter(lon[keepToo], lat[keepToo], s=10,
#                 c=np.clip(x_train[arg+1, :, days_pred-2], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
#                 vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

In [ ]:
fig = plt.figure(figsize=(30, 15))
ax = fig.add_subplot(121, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('big deviation')


sc = ax.scatter(lon[keep][index], lat[keep][index], s=10,
                c=np.clip(labels_train[arg, :][index], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

ax = fig.add_subplot(122, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('big deviation')


sc = ax.scatter(lon[keep][index], lat[keep][index], s=10,
                c=np.clip(res[arg, :][index], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

## Figures

In [ ]:
pathfig = './figures/'

In [ ]:
g_full = sphereGraph(lon, lat, 2, rad=False, epsilon=False)

In [ ]:
fig = plt.figure(figsize=(25,25))
ax = fig.add_subplot(111, projection='3d')
g_full.plot(vertex_size=10, edges=False, ax=ax)
ax.set_title('world weather stations', fontsize=20)
ax.view_init(azim=90)
# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# make the grid lines transparent
ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.axis('off')
plt.savefig(pathfig+"world_graph_USA.svg", bboxes_inches='tight')

In [ ]:
g_full.compute_laplacian("normalized")
g_full.compute_fourier_basis(recompute=True, n_eigenvectors=500)
g_full.set_coordinates(g_full.U[:,1:4])
g_full.plot(vertex_size=10)

In [ ]:
plt.plot(g_full.e[:16], 'o')

In [ ]:
us_lat = np.where(lat[keep_reg]>0)
us = [np.where(np.logical_or(lon[keep_reg][us_lat]<-30, 0))]

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.set_global()
# ax.stock_img()
ax.coastlines()

tmp = plt.plot(lon[keepToo][us_lat][us], lat[keepToo][us_lat][us], 'or', marker='o', markerfacecolor='r', markersize=2)

In [ ]:
neighbour = 8
gReg = sphereGraph(lon[keepToo][us_lat][us], lat[keepToo][us_lat][us], neighbour, rad=False, epsilon=False)
gReg.plot(vertex_size=10, edges=True)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
# gReg = sphereGraph(lon[keepToo][us_lat][us], lat[keepToo][us_lat][us], neighbour, rad=False, epsilon=False)
gReg.plot(vertex_size=10, edges=True, ax=ax)
ax.set_title('USA temperature stations', fontsize=20)
ax.view_init(elev=0, azim=70)
# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# make the grid lines transparent
ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.axis('off')

u, v = np.mgrid[0:2*np.pi:80j, 0:np.pi:40j]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)
ax.plot_surface(x, y, z, color="r", alpha=0.1)

plt.savefig(pathfig+"USA_temp.svg", bboxes_inches='tight')

In [ ]:
np.unravel_index(np.argmax(dataset_temp_reg), dataset_temp_reg.shape)

In [ ]:
timereg = np.empty_like(dataset_temp_reg[:,:,0])
timereg[:,:] = np.arange(timereg.shape[0])[:,np.newaxis]
plt.scatter(timereg[:,50], dataset_temp_reg[:,50,0], s=10)
plt.scatter(timereg[:,0], dataset_temp_reg[:,672,0], s=10)
plt.scatter(timereg[:,10], dataset_temp_reg[:,10,0], s=10)
plt.xlabel('day', fontsize=13)
plt.ylabel('Temperature [°C]', fontsize=13)
plt.savefig(pathfig+'evolution_temp.png', bboxes_inches='tight')

In [ ]:
plt.plot(w_days_cos*100)
plt.plot(dataset_prec[:,2,0], 'o', markersize=2)
plt.plot(dataset_prec[:,20,0], 'o', markersize=2)
plt.ylabel('precipitations [mm]', fontsize=15)
plt.xlabel('time [days]', fontsize=15)
# plt.plot(dataset_prec[:,100,0], 'o', markersize=2)
plt.savefig(pathfig+'prec_time.png', bboxes_inches='tight')

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection=ccrs.Orthographic(-50, 90))
ax.set_global()
ax.coastlines(linewidth=2)
ax.set_title('', fontsize=20)

zmin = -20
zmax = 30

sc = ax.scatter(lon[keepToo], lat[keepToo], s=20,
                c=np.clip(dataset_temp_reg[50, :, 1], zmin, zmax), cmap=plt.get_cmap('RdYlBu_r'),
                vmin=zmin, vmax=zmax, alpha=1, transform=ccrs.PlateCarree())

cbar = plt.colorbar(sc, ax=ax)
cbar.ax.tick_params(labelsize=30)

plt.savefig(pathfig+"temp_min.png", bboxes_inches='tight')

In [ ]:
if 'global' in EXP:
    plt.plot(labels_val, label='labels')
    plt.plot(res, label='predictions')
    plt.xlabel('day', fontsize=15)
    plt.ylabel('predicted day', fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(fontsize=13)
    # plt.ylim(top=1.1)
    plt.savefig(pathfig+'global_linear_results.png', bboxes_inches='tight')

In [ ]:
if 'future' in EXP:
    plt.scatter(timereg[days_pred:,50], np.asarray(predictions)[:,50], s=10)
    plt.scatter(timereg[days_pred:,0], np.asarray(predictions)[:,672], s=10)
    plt.scatter(timereg[days_pred:,10], np.asarray(predictions)[:,10], s=10)
    plt.savefig(pathfig+'prediction_temp.png', bboxes_inches='tight')

In [ ]:
if 'prec' in EXP:
# plt.scatter(np.atleast_3d(dataset_temp[:,:,1]), res)
# plt.scatter(labels_train*mask[:limit,:,1], res*mask[:limit,:,1])
plt.scatter(labels_train, res, s=2)
plt.plot([0, 100],[0, 100], 'r')
plt.xlabel('labels', fontsize=20)
plt.ylabel('predictions', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
# plt.plot(range(np.min(labels_train).astype(int), np.max(labels_train).astype(int)+1), 
#          range(np.min(labels_train).astype(int), np.max(labels_train).astype(int)+1), 'r')
plt.savefig(pathfig+'prec_reg_results.png', bboxes_inches='tight')